In [1]:
import time
import pandas as pd
from algosdk import account, mnemonic, logic
from algosdk.future import transaction
from algosdk.v2client import algod, indexer
from pyteal import compileTeal, Mode, Approve

ALGOD_TOKEN = "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"
TESTNET_ALGOD_RPC = "https://testnet-api.algonode.network"
TESTNET_INDEXER_RPC = "https://testnet-idx.algonode.network"

algod_client = algod.AlgodClient(ALGOD_TOKEN, TESTNET_ALGOD_RPC)
indexer_client = indexer.IndexerClient(ALGOD_TOKEN, TESTNET_INDEXER_RPC)
sp_func = algod_client.suggested_params
on_complete_param = transaction.OnComplete.NoOpOC
    
mnemonic_1 = open("../wallet_1").read().replace(',', ' ')
mnemonic_2 = open("../wallet_2").read().replace(',', ' ')
mnemonic_3 = open("../wallet_3").read().replace(',', ' ')

alice_private_key = mnemonic.to_private_key(mnemonic_1)
alice_address = account.address_from_private_key(alice_private_key)
bob_private_key = mnemonic.to_private_key(mnemonic_2)
bob_address = account.address_from_private_key(bob_private_key)
carol_private_key = mnemonic.to_private_key(mnemonic_3)
carol_address = account.address_from_private_key(carol_private_key)

print("Alice {} Balance: {}".format(alice_address, 
                algod_client.account_info(alice_address).get('amount') / 1e6))
print("Bob   {} Balance: {}".format(bob_address, 
                algod_client.account_info(bob_address).get('amount') / 1e6))
print("Carol {} Balance: {}".format(carol_address, 
                algod_client.account_info(carol_address).get('amount') / 1e6))

def submit_transaction(private_key: str, unsigned_txn: transaction.Transaction):
    signed_txn = unsigned_txn.sign(private_key)
    txid = algod_client.send_transaction(signed_txn)
    print("Signed transaction with txID: {}".format(txid))
    confirmed_txn = transaction.wait_for_confirmation(algod_client, txid, 3)
    print("Confirmed on round {}!".format(confirmed_txn['confirmed-round']))
    transaction_response = algod_client.pending_transaction_info(txid)
    return transaction_response

Alice GZ4IJXXNRFT23E6SLUOSSUWN2LUDFQTX4F6SXF5EP27LFWTOWHPFANLYIQ Balance: 36.581993
Bob   CIK3P7U4PZBJESWQ3XFDCZDXEZ3JZLWY6XZHY4A4KEJCA5SYYIPBT6W7Y4 Balance: 15.312008
Carol K7ZJP3J7SYYNB42DPZMRY56X32HVTYOOZRA2ASE2IND4LSQKCL3CE2Z2YU Balance: 8.977


In [2]:
import base64
from base64 import b64encode as en64
from base64 import b64decode as de64
import sys
from typing import List

def submit_transaction(private_key: str, unsigned_txn: transaction.Transaction):
    signed_txn = unsigned_txn.sign(private_key)
    txid = algod_client.send_transaction(signed_txn)
    print("Signed transaction with txID: {}".format(txid))
    confirmed_txn = transaction.wait_for_confirmation(algod_client, txid, 3)
    print("Confirmed on round {}!".format(confirmed_txn['confirmed-round']))
    transaction_response = algod_client.pending_transaction_info(txid)
    return transaction_response
    
def submit_transaction_group(private_key: str, unsigned_txns: List[transaction.Transaction]):
    gid = transaction.calculate_group_id(unsigned_txns)
    signed_txns = []
    for unsigned in unsigned_txns:
        unsigned.group = gid
        signed = unsigned.sign(private_key)
        signed_txns.append(signed)
    gtxid = algod_client.send_transactions(signed_txns)
    print("Signed transaction group with gtxID: {}".format(gtxid))
    confirmed_txn = transaction.wait_for_confirmation(algod_client, gtxid, 3)
    print("Confirmed on round {}!".format(confirmed_txn['confirmed-round']))
    transaction_response = algod_client.pending_transaction_info(gtxid)
    return transaction_response

def compile_program(client, source_code):
    compile_response = client.compile(source_code)
    return base64.b64decode(compile_response['result'])

blank_program = compile_program(algod_client, compileTeal(
    Approve(), Mode.Application, version=8
))

In [3]:
# Testnet Swap: 0100030a32c0c00000000000081cfd8e00000000000063f7572102ca2103c601
# PostSwap: https://mumbai.polygonscan.com/tx/0xbf2c3b77ff2b3928cf3258d14eccd3bbae2bcaa032b37eb04c3be7cd14f3f2c0
# https://testnet-explorer.meson.fi/swap/0x67d63e90804dedd3c6711ac19efe4eb8747e90fb92cd23140d233e42dd73b778

# 0x8302ce5a
# 0100030a32c0c00000000000081cfd8e00000000000063f7572102ca2103c601 (encodedSwap)
# 74659420d1810219f4633d6366efdcf410522ae32ce030382b07ecafea76b9da (r)
# 3fbc897918230c0dfdd87a7284e7bca0625bd8ff54f4bdd350d6cc82f1baedba (s)
# 000000000000000000000000000000000000000000000000000000000000001c (v)
# 000000000000002ef8a51f8ff129dbb874a0efb021702f59c1b2110000000001 (postingValue)

# assembly {
# mstore(0, encodedSwap)
# mstore(32, keccak256(0, 32))
# mstore(0, typehash)
# digest := keccak256(0, 64)
# }

# "bytes32 Sign to request a swap on Meson (Testnet)" (typehash-origin)
# 7b521e60f64ab56ff03ddfb26df49be54b20672b7acfffc1adeb256b554ccb25 (typehash)
# 8626c2a6698ce7518c71a8e6c3c4c8739ac5a799c97997198b73d4cf694be601 (hash(encodedSwap))
# bd045242342bc4e3948a5029209b0e90e29e5a55dffff09113aa65b8ea997031 (digest = hash(typehash, hash(encodedSwap)))

In [4]:
from pyteal import *

opup = OpUp(OpUpMode.Explicit, Int(1))

def get_pk(digest: Bytes, v: Bytes, r: Bytes, s: Bytes):
    pk = ScratchVar(TealType.bytes)
    return Seq(
        # opup.maximize_budget(Int(3000)),
        pk.store(EcdsaRecover(
                    EcdsaCurve.Secp256k1,
                    digest,
                    v, r, s
                ).outputReducer(lambda X, Y: Concat(X, Y))),
        App.globalPut(Bytes('result'), pk.load()),
        Approve()
    )

def ecdsa_try():
    return Cond(
        [Txn.application_id() == Int(0), Approve()],
        [Txn.on_completion() == OnComplete.OptIn, Approve(),],
        [Or(
            Txn.on_completion() == OnComplete.CloseOut,
            Txn.on_completion() == OnComplete.UpdateApplication,
            Txn.on_completion() == OnComplete.DeleteApplication,
        ), Reject()],
        [Txn.on_completion() == OnComplete.NoOp, Cond([
            Txn.application_args[0] == Bytes("verify"),
            get_pk(
                Txn.application_args[1], 
                Btoi(Txn.application_args[2]), 
                Txn.application_args[3], 
                Txn.application_args[4], 
            ),
        ], [
            Txn.application_args[0] == Bytes("nothing1"),
            Approve(),
        ], [
            Txn.application_args[0] == Bytes("nothing2"),
            Approve(),
        ], [
            Txn.application_args[0] == Bytes("nothing3"),
            Approve(),
        ])]
    )
    
ecdsa_program = compile_program(algod_client, teal_sentences := compileTeal(
    ecdsa_try(), Mode.Application, version=8
))

In [5]:
create_app_tx = submit_transaction(alice_private_key, transaction.ApplicationCreateTxn(
    alice_address, sp_func(), on_complete_param, ecdsa_program, blank_program,
    transaction.StateSchema(2, 2), transaction.StateSchema(0, 0)       # todo: add variable nums
))
print("Create Contract success! App id: %s, App Address: %s\n" % (
    app_index := create_app_tx['application-index'],
    app_address := logic.get_application_address(app_index)
))

Signed transaction with txID: UALXGNZVCJ3HCMUGDH3M6LUYC634SPWK7UUOEN3EO6SYDKTYO7SQ
Confirmed on round 28061939!
Create Contract success! App id: 161166710, App Address: DGH5EEGFXUMFS4VN5O4UDPFUVTT2WE35KXFHHEDQSEBNEUZ3FDSRZJPZZQ



In [6]:
# app_index = 160929623

digest = bytes.fromhex('bd045242342bc4e3948a5029209b0e90e29e5a55dffff09113aa65b8ea997031')
r_origin = bytes.fromhex('74659420d1810219f4633d6366efdcf410522ae32ce030382b07ecafea76b9da')
s_origin = bytes.fromhex('3fbc897918230c0dfdd87a7284e7bca0625bd8ff54f4bdd350d6cc82f1baedba')
v_origin = 1

submit_transaction_group(alice_private_key, [
    transaction.ApplicationCallTxn(
        alice_address, sp_func(), app_index, on_complete_param,
        app_args=['nothing1']
    ),
    transaction.ApplicationCallTxn(
        alice_address, sp_func(), app_index, on_complete_param,
        app_args=['nothing2']
    ),
    transaction.ApplicationCallTxn(
        alice_address, sp_func(), app_index, on_complete_param,
        app_args=['nothing3']
    ),
    transaction.ApplicationCallTxn(
        alice_address, sp_func(), app_index, on_complete_param,
        app_args=['verify', digest, v_origin, r_origin, s_origin]
    ),
])

Signed transaction group with gtxID: PCTG7PLOK5QDVRNAOS2VYGSSNL5DE7KDQKQQNFJYAFCMYE475OZA
Confirmed on round 28061941!


{'confirmed-round': 28061941,
 'pool-error': '',
 'txn': {'sig': 'YzpiZkTyVFsXgW4gS70aFN4bilse/Ott0EwoQrlSj+jdsrsCVZQQrq3H7n/mf2NCWKne6dDtAyo8NZgFX4aMBg==',
  'txn': {'apaa': ['bm90aGluZzE='],
   'apid': 161166710,
   'fee': 1000,
   'fv': 28061939,
   'gen': 'testnet-v1.0',
   'gh': 'SGO1GKSzyE7IEPItTxCByw9x8FmnrCDexi9/cOUJOiI=',
   'grp': 'M/IcPwk+oCStKD5vTuIODt9djwT00k7JNHVVnAju+CM=',
   'lv': 28062939,
   'snd': 'GZ4IJXXNRFT23E6SLUOSSUWN2LUDFQTX4F6SXF5EP27LFWTOWHPFANLYIQ',
   'type': 'appl'}}}

In [12]:
pk = de64('MhizErj41ZD7s9zSw9lsdPlbCOeu5DHreCJ3a6OIATKEjvNg0EFVU0texjuceG6W1FuC3jGw6TjwuOeoyEkDXw==')
pk.hex()

'3218b312b8f8d590fbb3dcd2c3d96c74f95b08e7aee431eb7822776ba3880132848ef360d04155534b5ec63b9c786e96d45b82de31b0e938f0b8e7a8c849035f'

In [58]:
'ba29fb34a5294f18510f0ce22ef8a51f8ff129dbb874a0efb021702f59c1b211'[-40:]

'2ef8a51f8ff129dbb874a0efb021702f59c1b211'

In [14]:
import ecdsa
private_key = bytes.fromhex('4719806c5b87c68e046b7b958d4416f66ff752ce60a36d28c0b9c5f29cbc9ab0')
sk = ecdsa.SigningKey.from_string(private_key, curve=ecdsa.SECP256k1)
vk = sk.get_verifying_key()
digest = bytes.fromhex('bd045242342bc4e3948a5029209b0e90e29e5a55dffff09113aa65b8ea997031')
sig = sk.sign(digest)
vk.verify(sig, digest)
r, s = sig[:32], sig[32:]
v = int(s[0] & 128 != 0)

In [15]:
r, s, v

(b"z\xd3\x11\xea\xb0\x94\xc2\xba8\xb7G\xfc\x9cn\xca\x1b\xd2\xa3K(\xaa\x1b'\xd5\x87\xdbQ\xfe\xfd#\x02\xff",
 b'<\x1c\xab\xe6\xd5G\xee:u\x86\xc8\t\x94\x0bl\xbf\xb4\xc2~s\xfa\xd5\xdfY\x1a\xaf+6\xf85\x16\xae',
 0)

In [16]:
submit_transaction_group(alice_private_key, [
    transaction.ApplicationCallTxn(
        alice_address, sp_func(), app_index, on_complete_param,
        app_args=['verify', digest, v, r, s]
    ),
    transaction.ApplicationCallTxn(
        alice_address, sp_func(), app_index, on_complete_param,
        app_args=['nothing1']
    ),
    transaction.ApplicationCallTxn(
        alice_address, sp_func(), app_index, on_complete_param,
        app_args=['nothing2']
    ),
    transaction.ApplicationCallTxn(
        alice_address, sp_func(), app_index, on_complete_param,
        app_args=['nothing3']
    ),
])

Signed transaction group with gtxID: 6A4CRC2FXPM4LSLUAUVLXE3D3Y62JOMB3AJSLOOHMKYHZAJO4NUA
Confirmed on round 28061988!


{'confirmed-round': 28061988,
 'global-state-delta': [{'key': 'cmVzdWx0',
   'value': {'action': 1,
    'bytes': '6vzYSD1eO4k97MLrJybOI3vWDRIlNGkldnKCL4zGO7h132fjPCWE1p55OMp6b8C0t6TzvxJBJkEVmNFvqr91qQ=='}}],
 'pool-error': '',
 'txn': {'sig': 'sFupHhrf6+qDiDy+1LtkvqgwD0aszAzm0A1kfhbdvkds7CEkGMlXP5sJ89Wync2eJvyNYEOH1zWLaqf48Eo8DQ==',
  'txn': {'apaa': ['dmVyaWZ5',
    'vQRSQjQrxOOUilApIJsOkOKeWlXf//CRE6pluOqZcDE=',
    'AAAAAAAAAAE=',
    'etMR6rCUwro4t0f8nG7KG9KjSyiqGyfVh9tR/v0jAv8=',
    'PByr5tVH7jp1hsgJlAtsv7TCfnP61d9ZGq8rNvg1Fq4='],
   'apid': 161166710,
   'fee': 1000,
   'fv': 28061985,
   'gen': 'testnet-v1.0',
   'gh': 'SGO1GKSzyE7IEPItTxCByw9x8FmnrCDexi9/cOUJOiI=',
   'grp': 'OS3u6d5xa2eLCDVt0b7VcBEOLcwz5xGuZkD0jxracOQ=',
   'lv': 28062985,
   'snd': 'GZ4IJXXNRFT23E6SLUOSSUWN2LUDFQTX4F6SXF5EP27LFWTOWHPFANLYIQ',
   'type': 'appl'}}}

In [10]:
de64('NzOs8mbcAZAduzORfbC2Qu79KDhBUnjP8H7Som+qfP4Yxuhjlr/aYfwIQtFSBgcUx0GtOVaNmpY4SxmKMTv3jg==').hex()

'3733acf266dc01901dbb33917db0b642eefd2838415278cff07ed2a26faa7cfe18c6e86396bfda61fc0842d152060714c741ad39568d9a96384b198a313bf78e'

In [17]:
de64('6vzYSD1eO4k97MLrJybOI3vWDRIlNGkldnKCL4zGO7h132fjPCWE1p55OMp6b8C0t6TzvxJBJkEVmNFvqr91qQ==').hex()

'eafcd8483d5e3b893decc2eb2726ce237bd60d12253469257672822f8cc63bb875df67e33c2584d69e7938ca7a6fc0b4b7a4f3bf124126411598d16faabf75a9'

In [17]:
de64('MhizErj41ZD7s9zSw9lsdPlbCOeu5DHreCJ3a6OIATKEjvNg0EFVU0texjuceG6W1FuC3jGw6TjwuOeoyEkDXw==').hex()

'3218b312b8f8d590fbb3dcd2c3d96c74f95b08e7aee431eb7822776ba3880132848ef360d04155534b5ec63b9c786e96d45b82de31b0e938f0b8e7a8c849035f'